In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Downloading files

In [ ]:
# !wget --header="Host: www.ncbi.nlm.nih.gov" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-GB,en;q=0.9,en-US;q=0.8,bn;q=0.7" --header="Referer: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE214611" --header="Cookie: _gid=GA1.2.843222926.1683780721; ncbi_sid=0C4234AB45C74711_0000SID; _ga=GA1.1.1244755766.1683780721; _ga_DP2X732JSX=GS1.1.1683780720.1.1.1683782404.0.0.0; _gat_ncbiSg=1; ncbi_pinger=N4IgDgTgpgbg+mAFgSwCYgFwgAwGEAsATAMwDsAjAGz4Csup+pl52rb7H5IANCAMYAbZHwDWAOygAPAC6ZQxAJyYQVUkwAc2HiHxasqhfirb8+ZYOHips3rWVdblZQDMAhgIDOUE6WXUttur25PiUAHSUkZTqpNjk5BTqQbZKWNo0eiCI0tJgHhgA9AUA7qVhYnwARsjlAgC25ciIYQDmAPYwBS1QbQWobcViAm2uqAUA/K58fAC8AOIAygCihCHM5ABkzm0Qda7SM87IAt68NFxpZ4TKABTZuQCUIAC+vACuQyOoEjJyIMTEFzuLzaYhmLDSCBvU7/IKXf6pFSUNSUQwmTIGIhKWzgkAfYajH42HQ0ZTXM5OLA0VHpXxYAIgZjKTSA178Np1OptMREv7XLDdNqg5QmFxvAQCdIi3iETJ4IhkKi0eiMZgcdXsBwgVbKQUYAByAHl9UsMM5xQIDcaltpCPyQKViuUqjUhg0xE1Wh1bYjyApZaCMdhNKCLip1Phkv97SwaIDbID6WFiOowuSQHH7CR0zRcdTfM9nkA" --header="Connection: keep-alive" "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE214611&format=file" -c -O '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar'

In [ ]:
# !tar -xvf '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW.tar' -C '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/'

In [ ]:
# !cd '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/' && for f in *.tar.gz; do tar -xvzf "$f" -C "$(dirname "$f")"; done

In [ ]:
# !unzip '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW/Single-Cell-Spatial-Transcriptomics-for-Border-zone-BZ_Cell_Mapping.zip' -d '/content/drive/MyDrive/Samin/PathologicalCells/data/GSE214611_RAW'

## scPSS

In [ ]:
!pip install git+https://github.com/SaminRK/scPSS.git
!pip install scvi-tools

  Cloning https://github.com/SaminRK/scPSS.git to /tmp/pip-req-build-_km_4imx
  Running command git clone --filter=blob:none --quiet https://github.com/SaminRK/scPSS.git /tmp/pip-req-build-_km_4imx
  Resolved https://github.com/SaminRK/scPSS.git to commit 28c1e6d9a4bbc44848e47d23d7221a29e30e8820
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.0 MB/s eta 0:00:00
  Created wheel for scPSS: filename=scpss-0.1.1-py3-none-any.whl size=7225 sha256=a7fe23653eff54025b02d73e49e1dd63d940fec09900c96bdda805ce900a4939
  Stored in directory: /tmp/pip-ephem-wheel-cache-uszkyzs7/wheels/38/a1/b0/2c6e36cd72b420c8578046e45e829c1

In [ ]:
## Modify to the source directory of single cell data
ROOT_PATH = '/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/data/Infracted Heart Border Zone - GSE214611_RAW'

In [ ]:
import scanpy as sc
import pandas as pd

In [ ]:
metadata_sc = pd.read_csv(f'{ROOT_PATH}/Single-Cell-Spatial-Transcriptomics-for-Border-zone/sn_wc_object_integrated@meta.data.csv', index_col=0)
metadata_sc_t = metadata_sc.rename(index=lambda x: x.split('_')[0])
metadata_sc_t['obs_names'] = metadata_sc_t.index

In [ ]:
def get_anndata(mtx_path_id, metadata_orig_id):
    PATH = f'{ROOT_PATH}/{mtx_path_id}'
    ad = sc.read_10x_mtx(PATH)
    ad.obs = ad.obs.rename(index=lambda x: x.split('-')[0])
    A = metadata_sc_t[metadata_sc_t['orig.ident'] == metadata_orig_id]

    ad.obs = ad.obs.merge(A, left_index=True, right_index=True, how='left')

    return ad

In [ ]:
def pp_anndata(ad, n_top_genes=None, plot=False):
    ad.var_names_make_unique()

    sc.pp.filter_cells(ad, min_genes=200)
    sc.pp.filter_genes(ad, min_cells=3)

    ad.var['mt'] = ad.var_names.str.startswith('mt-')
    sc.pp.calculate_qc_metrics(ad, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    if plot:
        sc.pl.violin(ad, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
                jitter=0.4, multi_panel=True)
        sc.pl.scatter(ad, x='total_counts', y='pct_counts_mt')
        sc.pl.scatter(ad, x='total_counts', y='n_genes_by_counts')

    ad = ad[ad.obs.pct_counts_mt <= 5, :]

    return ad

## Prepare training and testing dataset

In [ ]:
def get_mtx_path_id_metadata_orig_id(problem_key):
    exp_meta_keys = {
        'ref_0_que_1hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
            ]
        },
        'ref_0_que_4hr': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn4hr', '4HR'),
            ]
        },
        'ref_0_que_1': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
            ]
        },
        'ref_0_que_3': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
            ]
        },
        'ref_0_que_7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        },
        'ref_0_que_1-7': {
            'reference': [
                ('snd0_1', 'D0'),
                ('snd0_2', 'D0'),
                ('snd0_3', 'NoMI_Nuclei'),
            ],
            'query': [
                ('sn1hr_1', 'sn1_1'),
                ('sn1hr_2', 'sn1_2'),
                ('sn4hr', '4HR'),
                ('snd1_1', 'D1_MI'),
                ('snd1_2', 'D1_MI_Hrt_nuclei'),
                ('snd1_3', 'D1_IR30_Hrt_nuclei'),
                ('snd1_4', 'WT_IR30_D1_2'),
                ('snd1_5', 'WT_IR30_D1_3'),
                ('snd3_1', 'WT_IR30_D3'),
                ('snd3_2', 'D3F_M_rep1'),
                ('snd3_3', 'D3F_M_rep2'),
                ('snd7_1', 'snD7_1'),
                ('snd7_2', 'snD7_1'),
                ('snd7_3', 'D5_D7'),
            ]
        }
    }

    return exp_meta_keys[problem_key]

In [ ]:
import anndata as AD

imp_celltypes = ['Ankrd1', 'Xirp2', 'Myh6']


def get_ad(PROBLEM_KEY):

    exp_meta_keys = get_mtx_path_id_metadata_orig_id(PROBLEM_KEY)

    adatas = {}

    for key, item in exp_meta_keys.items():
        for i, (mtx_path_id, metadata_orig_id) in enumerate(item):
            ad = pp_anndata(get_anndata(mtx_path_id, metadata_orig_id))
            ad = ad[ad.obs['final_cluster'].isin(imp_celltypes)]
            dataset_id = f"{key}_{i+1}"
            ad.obs['sample'] = dataset_id
            ad.obs['dataset'] = key
            adatas[f"{key}_{i+1}"] = ad

    ad = AD.concat(adatas, label='batch_key')
    ad.obs_names_make_unique()

    cluster_to_zone = {
        "Ankrd1": "BZ1",
        "Xirp2": "BZ2",
        "Myh6": "RZ",
    }

    ad.obs["zone"] = ad.obs["final_cluster"].map(cluster_to_zone)

    return ad

In [ ]:
## Modify to the directory where outputs should be stored
OUT_PATH = "/content/drive/MyDrive/Research - Sohel sir/Pathological Cells/Pathological Cell Identification/reproducibility/outputs"
metrics_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-metrics-scvi-2.csv"
results_csv_path = f"{OUT_PATH}/scpss-mouse-infarcted-heart-results-scvi-2.csv"

In [ ]:
from scpss import scPSS
import time
import numpy as np


with open(metrics_csv_path, 'w') as f:
    print('PROBLEM_KEY', 'seed', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'aupr', 'outlier_ratio_ref', 'outlier_ratio_que', 'time_taken',  sep=',', file=f, flush=True)

    # 'ref_0_que_1hr', 'ref_0_que_4hr', 'ref_0_que_1',
    for PROBLEM_KEY in [ 'ref_0_que_3', 'ref_0_que_7',  'ref_0_que_1-7']:
        ad = get_ad(PROBLEM_KEY)

        reference_samples = list(ad.obs[ad.obs['dataset'] == 'reference']['sample'].unique())
        query_samples = list(ad.obs[ad.obs['dataset'] == 'query']['sample'].unique())


        for i in range(25):
            print('RUN: ', i)
            seed = i * 100

            start_time = time.perf_counter()

            scpss = scPSS(ad.copy(), 'sample', reference_samples, query_samples)

            import scvi

            scvi.model.SCVI.setup_anndata(scpss.adata, batch_key='sample')
            model = scvi.model.SCVI(scpss.adata)
            model.train(early_stopping=True)

            scpss.adata.obsm["X_scVI"] = model.get_latent_representation()
            scpss.__obsm_str__ = 'X_scVI'

            scpss.find_optimal_parameters()
            scpss.set_distance_and_condition()

            end_time = time.perf_counter()

            ad_que = scpss.adata[scpss.query_mask]

            true_labels = ad_que.obs['zone'].isin(['BZ1', 'BZ2']).to_numpy()
            predicted_labels = ad_que.obs['scpss_condition'].isin(['diseased']).to_numpy()
            dist_que_ref = scpss.adata.obs.loc[scpss.query_mask, 'scpss_distances']
            dist_ref_ref = scpss.adata.obs.loc[scpss.reference_mask, 'scpss_distances']

            time_taken = end_time - start_time


            from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

            accuracy = accuracy_score(true_labels, predicted_labels)
            precision = precision_score(true_labels, predicted_labels)
            recall = recall_score(true_labels, predicted_labels)
            f1 = f1_score(true_labels, predicted_labels)

            print("Accuracy:", accuracy)
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1-score:", f1)

            from sklearn.metrics import roc_auc_score, average_precision_score
            from sklearn.metrics import roc_curve, precision_recall_curve

            y_true = true_labels
            y_scores = dist_que_ref

            auc = roc_auc_score(y_true, y_scores)
            print("AUC:", auc)

            aupr = average_precision_score(y_true, y_scores)
            print("AUPR:", aupr)

            outlier_ratio_ref = np.mean(dist_ref_ref > scpss.best_params['threshold'])
            print("Outlier ratio in reference:", outlier_ratio_ref)
            outlier_ratio_que = np.mean(dist_que_ref > scpss.best_params['threshold'])
            print("Outlier ratio in query:", outlier_ratio_que)


            print(PROBLEM_KEY, seed, accuracy, precision, recall, f1, auc, aupr, outlier_ratio_ref, outlier_ratio_que, time_taken, sep=',', file=f, flush=True)


<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample'] = dataset_id
<ipython-input-16-e729ee5316fc>:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sample']

RUN:  0


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training:   0%|          | 0/400 [00:00<?, ?it/s]

Monitored metric elbo_validation did not improve in the last 45 records. Best score: 6227.925. Signaling Trainer to stop.
✅ Found Optimal Parameters.


...

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training:   0%|          | 0/400 [00:00<?, ?it/s]

Monitored metric elbo_validation did not improve in the last 45 records. Best score: 1643.907. Signaling Trainer to stop.
✅ Found Optimal Parameters.
✅ Stored distances and conditions in Anndata object.
Accuracy: 0.6462745098039215
Precision: 0.9130691898285038
Recall: 0.2829912023460411
F1-score: 0.43206939974814607
AUC: 0.83122053258523
AUPR: 0.8154375624757471
Outlier ratio in reference: 0.0671912832929782
Outlier ratio in query: 0.14736383442265796
RUN:  17


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training:   0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
metrics = pd.read_csv(metrics_csv_path)

In [ ]:
metrics

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t

# Confidence level
confidence = 0.95

# Function to calculate mean ± margin
def mean_margin(series, confidence=0.95):
    n = len(series)
    mean = np.mean(series)
    se = sem(series)
    margin = se * t.ppf((1 + confidence) / 2., n - 1)
    return f"{mean:.4f} ± {margin:.4f}"

# Group by 'Group' and calculate mean ± margin for other columns
grouped = metrics.groupby('PROBLEM_KEY')

result = grouped.agg({
    col: lambda x: mean_margin(x, confidence)
    for col in metrics.columns if col != 'PROBLEM_KEY'
})

# Reset index to make it look cleaner
result.reset_index(inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(results_csv_path, index=False)